#  运行来自https://github.com/eiriniar/gleason_CNN.git的模型

# connect to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 更改工作目录到包含你的Python脚本的文件夹
import os
os.chdir('/content/drive/MyDrive/auto_gleason')

In [6]:
!pip install keras==2.2.4
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

# 1 准备数据集：

将您的图像按照项目中指定的结构组织好：

```
/path/to/your_dataset
|-- TMA_images                            # 您的H&E染色图像
|-- Gleason_masks_train                   # 训练和验证的Gleason标注
|-- Gleason_masks_test                    # 测试集的Gleason标注

```



# 2 创建图像补丁：
	•	使用 utils/create_patches.py 脚本为训练和验证数据创建图像补丁。

In [3]:

!python utils/create_patches.py --input_dir /content/drive/MyDrive/auto_gleason/input/slide001_core003.png --output_dir /content/drive/MyDrive/auto_gleason/output


/content/drive/MyDrive/auto_gleason/utils/create_patches.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(img, mode='RGB')


# 3 训练模型或加载预训练模型：

您可以选择自己训练模型，也可以使用提供的预训练模型。如果选择自己训练模型，请运行 gleason_score_finetune.py 脚本：

In [ ]:

!python gleason_score_finetune.py --train_dir /path/to/patches_output --val_dir /path/to/patches_output --model_output /path/to/saved_model


如果使用预训练模型，确保将 model_weights/MobileNet_gleason_weights.h5 下载并放置在合适的目录中。

# 4 创建组织区域掩码

使用 utils/create_tissue_masks.py 脚本为您的TMA图像创建组织区域掩码：

In [4]:

!python utils/create_tissue_masks.py --input_dir /content/drive/MyDrive/auto_gleason/input/slide001_core003.png --output_dir /content/drive/MyDrive/auto_gleason/output


这将生成组织区域掩码，并将结果保存在 tissue_masks_output 目录中。


# 5 生成像素级概率映射并可视化：

使用训练好的模型对整个TMA点生成像素级概率映射，并通过类激活映射（CAM）可视化类判别区域，运行 plot_heatmaps_and_CAM.py 脚本：

In [5]:

!python /content/drive/MyDrive/auto_gleason/plot_heatmaps_and_CAM.py --input_dir /content/drive/MyDrive/auto_gleason/input/slide001_core003.png --mask_dir /content/drive/MyDrive/auto_gleason/output --model_path /content/drive/MyDrive/auto_gleason/MobileNet_Gleason_weights.h5 --output_dir /content/drive/MyDrive/auto_gleason/output


2024-06-13 17:56:50.702149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 17:56:50.702249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 17:56:50.706024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 17:56:50.719965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 17:56:51.891346: W tensorflow/compiler/tf2

# 6 比较模型和病理学家在测试集上的注释：

使用 plot_test_cohort_results.ipynb Jupyter Notebook，对模型和病理学家在测试集上的格利森注释进行比较：